# 02 — Preprocessing & Feature Engineering

> **Objective:** To implement the cleaning and feature-engineering steps used in the overqualification pipeline: handling NGS special codes, normalizing mixed-type columns, and preparing categorical features for CatBoost.

This notebook covers:
1. [**Preprocessing**](#preprocessing) — `clean()`: missing codes and categorical normalization  
2. [**Feature engineering**](#feature-engineering) — `add_features()`: categorical encoding for CatBoost  
3. [**Before/after comparison**](#before-and-after-comparison) — data shape and sample values

### 🧠 Context

The NGS dataset uses **6, 9, 99** as valid skip / refused / not stated. The pipeline treats these as missing and fills them consistently. Columns such as **GENDER2**, **DDIS_FL**, and **VISBMINP** sometimes contain text (e.g. "Female", "With disability") in addition to numeric codes; we normalize these to numeric codes before converting to categorical strings for CatBoost.

---
### 🧰 Imports

In [ ]:
import sys
from pathlib import Path

import pandas as pd

sys.path.insert(0, str(Path().resolve().parent))

from src.data import load_train
from src.preprocess import clean
from src.features import add_features, get_categorical_feature_names

### 📥 Load raw data

In [ ]:
df_raw = load_train()
print("Shape:", df_raw.shape)
df_raw.head()

### 🧹 Preprocessing <a id="preprocessing"></a>

`clean()`:
- Replaces NGS codes **6, 9, 99** with `NaN` in numeric/code columns  
- Normalizes **GENDER2** (e.g. "Male" → 1, "Female" → 2)  
- Normalizes **DDIS_FL** ("With disability" / "Without disability")  
- Normalizes **VISBMINP** (e.g. "Yes" / "No")

In [ ]:
df_cleaned = clean(df_raw)
print("After clean():")
print("  GENDER2 sample values:", df_cleaned["GENDER2"].dropna().astype(str).unique()[:8])
print("  DDIS_FL sample values:", df_cleaned["DDIS_FL"].dropna().astype(str).unique()[:8])
print("  Null count (should increase where 6/9/99 were replaced):", df_cleaned.isnull().sum().sum())

### 🔧 Feature Engineering <a id="feature-engineering"></a>

`add_features()`:
- Converts all survey-code columns to **string** type (CatBoost treats object columns as categorical)  
- Fills remaining NaN in those columns with the string `"missing"` so CatBoost can use them as a category

In [ ]:
df_engineered = add_features(df_cleaned)
cat_cols = get_categorical_feature_names()
print("Categorical feature names (for CatBoost):", cat_cols)
print("\nSample of engineered columns (string type):")
print(df_engineered[cat_cols[:5]].dtypes)
print(
    "\nUnique values in CERTLEVP (after add_features):",
    df_engineered["CERTLEVP"].astype(str).unique()[:10],
)

### 📊 Before and After Comparison <a id="before-and-after-comparison"></a>

In [ ]:
print("Raw shape:", df_raw.shape)
print("After clean + add_features:", df_engineered.shape)
print("\nNo rows/columns dropped; only types and values normalized.")
print("\nPipeline order: load_train() → clean() → add_features() → split_X_y() for model.")

---
## 📝 Summary

Preprocessing and feature engineering produce a single DataFrame that retains `id` and `overqualified` and has all predictor columns as **string-typed categories** suitable for CatBoost. The same sequence is used in `src/train.py` and `src/predict.py`.

**Next step:** `03_catboost_training_tuning.ipynb` — train and tune the CatBoost model.